<a href="https://colab.research.google.com/github/Tecnologico-de-Monterrey-MNA/iayaa-Equipo-19/blob/main/SVD_Equipo19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Maestría en Inteligencia Artificial Aplicada**
##**Curso: Inteligencia Artificial y Aprendizaje Automático**
###Tecnológico de Monterrey
###Prof Luis Eduardo Falcón Morales

## **Adtividad de la Semana 10**
###**Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Nombres y matrículas de los integrantes del equipo:**

* Aurelio Antonio Lozano Rabago A01081266  
* Josias Ruiz Peña A00968460  
* David González A01794025  
* Jerson David Pérez Contreras A01793810  
* Jose Alberto Mtanous Treviño A00169781  

En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [ ]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.
import pandas as pd
import numpy as np
import requests, zipfile
from io import BytesIO

from sklearn.decomposition import TruncatedSVD

#### Ejercicio-1.
Descarga los archivos y genera la matriz de utilidad en la cual los renglones son los usuarios 
(user_ID), las columnas los nombres de los restaurantes (name) y las entradas la evaluaciones 
generales dadas por los usuarios (rating). En particular indica el tamaño de dicha matriz y despliega 
sus primeros 10 renglones.  

In [ ]:
path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00232/RCdata.zip'
ratingFileName = '/tmp/rating_final.csv'
geoplacesFileName = '/tmp/geoplaces2.csv'

req = requests.get(path)
zp = zipfile.ZipFile(BytesIO(req.content))
zp.extractall(path='/tmp/')

ratingDf = pd.read_csv(ratingFileName)
geoplacesDf = pd.read_csv(geoplacesFileName, encoding = "latin-1")

In [ ]:
geoplacesDf.columns

Index(['placeID', 'latitude', 'longitude', 'the_geom_meter', 'name', 'address',
       'city', 'state', 'country', 'fax', 'zip', 'alcohol', 'smoking_area',
       'dress_code', 'accessibility', 'price', 'url', 'Rambience', 'franchise',
       'area', 'other_services'],
      dtype='object')

In [ ]:
ratingDf.columns

Index(['userID', 'placeID', 'rating', 'food_rating', 'service_rating'], dtype='object')

In [ ]:
# Shape, NANs, Info
print(ratingDf.shape)
ratingDf.isna().sum()/ratingDf.shape[0]

(1161, 5)


userID            0.0
placeID           0.0
rating            0.0
food_rating       0.0
service_rating    0.0
dtype: float64

In [ ]:
# Shape, NANs, Info
print(geoplacesDf.shape)
geoplacesDf.isna().sum()/geoplacesDf.shape[0]

(130, 21)


placeID           0.0
latitude          0.0
longitude         0.0
the_geom_meter    0.0
name              0.0
address           0.0
city              0.0
state             0.0
country           0.0
fax               0.0
zip               0.0
alcohol           0.0
smoking_area      0.0
dress_code        0.0
accessibility     0.0
price             0.0
url               0.0
Rambience         0.0
franchise         0.0
area              0.0
other_services    0.0
dtype: float64

In [ ]:
ratingDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1161 entries, 0 to 1160
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userID          1161 non-null   object
 1   placeID         1161 non-null   int64 
 2   rating          1161 non-null   int64 
 3   food_rating     1161 non-null   int64 
 4   service_rating  1161 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 45.5+ KB


In [ ]:
geoplacesDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   placeID         130 non-null    int64  
 1   latitude        130 non-null    float64
 2   longitude       130 non-null    float64
 3   the_geom_meter  130 non-null    object 
 4   name            130 non-null    object 
 5   address         130 non-null    object 
 6   city            130 non-null    object 
 7   state           130 non-null    object 
 8   country         130 non-null    object 
 9   fax             130 non-null    object 
 10  zip             130 non-null    object 
 11  alcohol         130 non-null    object 
 12  smoking_area    130 non-null    object 
 13  dress_code      130 non-null    object 
 14  accessibility   130 non-null    object 
 15  price           130 non-null    object 
 16  url             130 non-null    object 
 17  Rambience       130 non-null    obj

In [ ]:
# Diferentes columnas entre data sets
print(set(geoplacesDf.columns).difference(set(ratingDf.columns)))
print(set(ratingDf.columns).difference(set(geoplacesDf.columns)))

{'the_geom_meter', 'price', 'city', 'area', 'franchise', 'zip', 'Rambience', 'alcohol', 'dress_code', 'fax', 'longitude', 'name', 'state', 'smoking_area', 'address', 'url', 'other_services', 'country', 'accessibility', 'latitude'}
{'userID', 'service_rating', 'rating', 'food_rating'}


In [ ]:
# Columnas compartidas entre data sets
set(geoplacesDf.columns) & (set(ratingDf.columns))

{'placeID'}

In [ ]:
rating_lab = ['placeID','userID','rating']
geoplaces_lab = ['placeID','name']

X = pd.merge(ratingDf[rating_lab],geoplacesDf[geoplaces_lab], on='placeID')
print('------------Tamaño de Matriz--------------------')
print(X.shape)
print('------------Primeros 10 renglones--------------------')
X.head(10)

------------Tamaño de Matriz--------------------
(1161, 4)
------------Primeros 10 renglones--------------------


,placeID,userID,rating,name
0,135085,U1077,2,Tortas Locas Hipocampo
1,135085,U1108,1,Tortas Locas Hipocampo
2,135085,U1081,1,Tortas Locas Hipocampo
3,135085,U1056,2,Tortas Locas Hipocampo
4,135085,U1134,2,Tortas Locas Hipocampo
5,135085,U1116,2,Tortas Locas Hipocampo
6,135085,U1132,2,Tortas Locas Hipocampo
7,135085,U1120,0,Tortas Locas Hipocampo
8,135085,U1036,2,Tortas Locas Hipocampo
9,135085,U1084,2,Tortas Locas Hipocampo


#### Ejercicio-2.
Con el objetivo de obtener la información con respecto a los vectores latentes de los restaurantes, 
obtener dicha factorización SVD utilizando todas las componentes posibles, es decir, usando todos 
sus valores singulares. En particular indica la cantidad de valores singulares obtenidos. 

In [ ]:
#X = pd.merge(ratingDf, geoplacesDf, how="inner", on="placeID")[['userID', 'name', 'rating']]
utilityMatrixEx1 = X.pivot_table(values='rating', index='userID', columns='name', fill_value=0)
print('------------Tamaño de Matriz--------------------')
print(utilityMatrixEx1.shape)
print('------------Primeros 10 renglones--------------------')
utilityMatrixEx1.head(10)

------------Tamaño de Matriz--------------------
(138, 129)
------------Primeros 10 renglones--------------------


name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,1,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
# Como queremos obtener información sobre las restuarantes, tenemos que dejar los renglones con los restaurantes y las columnas con los usuarios, para esto pudemos usar la matriz transpuesta
utilityMatrixEx2T = utilityMatrixEx1.T
print(utilityMatrixEx2T.shape)
utilityMatrixEx2T.head()

(129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Arrachela Grill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cabana Huasteca,0,0,2,0,0,0,0,0,1,0,...,0,0,0,0,0,2,0,0,0,0
Cafe Chaires,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cafeteria cenidet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
print('------------El Numero de componentes son--------------------')
print(utilityMatrixEx2T.shape[1])

------------El Numero de componentes son--------------------
138


In [ ]:
SVD = TruncatedSVD(n_components=137)
SVD.fit(utilityMatrixEx2T)
print('------------El Numero de Valores singulares es--------------------')
print(SVD.singular_values_.shape[0])

------------El Numero de Valores singulares es--------------------
129


In [ ]:
print('Primeros 10 vectores singulares:')
SVSSum=round((SVD.explained_variance_ratio_[0:10]).sum()*100,2)
print(SVSSum,'%')

print('Información reducida con 10 vectores singulares:')
print(100-SVSSum,'%')

Primeros 10 vectores singulares:
43.49 %
Información reducida con 10 vectores singulares:
56.51 %


#### Ejercicio-3.
Usando la métrica de correlación de Pearon[sic], obtener las 10 mejores recomendaciones de este 
modelo no supervisado con base a la información de alguien que evaluó bien de manera general al 
“Subway”. Despliega también el valor numérico de la correlación de Pearson de cada una de estas 10 
recomendaciones. NOTA: No debe estar incluido “Subway” dentro de las recomendaciones. 

In [ ]:
corrPearson = np.corrcoef(utilityMatrixEx2T, )

In [ ]:
# Ahora encontramos el número de renglón para poder ver la correlación con los otros restaurantes
RestLabel = 'Subway'
restaurantes = list(utilityMatrixEx1.columns)

In [ ]:
# Creamos un dataframe con en nombre del Restaurante y la correlación al Subway
Corr = corrPearson[restaurantes.index(RestLabel)]
sTmp = Corr.reshape((-1,1))
rTmp = np.array(restaurantes).reshape((-1,1))
aTmp = np.concatenate( (rTmp,sTmp),axis=1 )
CorrLabel = 'Corr_'+RestLabel
CorrDf = pd.DataFrame(aTmp,columns=['Restaurante',CorrLabel])
CorrDf.head(2)

,Restaurante,Corr_Subway
0,Abondance Restaurante Bar,-0.047574119740737185
1,Arrachela Grill,-0.025652506155829518


In [ ]:
# Ahora encontraremos las 10 correlaciones más altas sin incluir Subway:
firstTenRecomendationsEx2 = CorrDf.sort_values(by=CorrLabel,ascending=False)[1:11]
firstTenRecomendationsEx2

,Restaurante,Corr_Subway
58,Restaurant Teely,0.5990533791287092
97,Vips,0.479864116498316
85,Rincon del Bife,0.4093707292614752
60,Restaurant and Bar and Clothesline Carlos N Ch...,0.40937072926147494
86,Sanborns Casa Piedra,0.38696844626865706
41,Mariscos Tia Licha,0.35446853122212385
45,Mikasa,0.284129903907283
11,Chilis Cuernavaca,0.28412990390728293
64,Restaurant los Pinos,0.2622531234331326
37,Log Yin,0.20832864690144098


#### Ejercicio-4.
Disminuye ahora la dimensión de la factorización con un número de componentes tal, que las 
recomendaciones con esta menor cantidad de información coincidan en al menos 9 de las 10 
recomendaciones dadas con todos los valores singulares. Indica la cantidad de valores singulares 
encontrado y la cantidad de información que contienen. 

In [ ]:
# Para encontrar la cantidad de valores singulars mínima para que coincidan 9 de 10 recomendaciones voy a iterar y revisar los valores
# de cada iteración hasta que tenga 2 de 130
RestLabel = 'Subway'
CorrLabel = 'Corr_'+RestLabel

# Ahora vamos a descomponer la matriz de utilidad en sus valores singulares
DFCompare=firstTenRecomendationsEx2
UtilityMatrix=utilityMatrixEx2T

num_sv = 0
firstTenRecomendations = pd.DataFrame()
SVD = None

for n in range(2,130):

    SVD = TruncatedSVD(n_components=n, n_iter=1000, random_state=8)
    
    reducedUtilityMatrix = SVD.fit_transform(UtilityMatrix)
    #Ahora calculamos la matriz de correlación usando la matriz reconstruida
    reducedUtilityMatrixCorr = np.corrcoef(reducedUtilityMatrix, )
    
    Corr = reducedUtilityMatrixCorr[restaurantes.index(RestLabel)]
    
    sTmp = Corr.reshape((-1,1))
    rTmp = np.array(restaurantes).reshape((-1,1))
    aTmp = np.concatenate( (rTmp,sTmp),axis=1 )
    
    CorrDf = pd.DataFrame(aTmp,columns=['Restaurante',CorrLabel])
    
    firstTenRecomendations = CorrDf.sort_values(by=CorrLabel,ascending=False)[1:11]
    
    strikes = 0
    for idx,name in firstTenRecomendations['Restaurante'].items():
        if name not in DFCompare['Restaurante'].unique():
            strikes += 1
        if strikes > 1:
            break
    if strikes == 1:
        num_sv = n
        break

print('----------------El Número de valores singulares mínimo para encontrar 9 de las 10 recomendaciones es:-----------------------')
print(num_sv)
print('-------------Las recomendaciones encontradas son-----------------------')
for idx,name in firstTenRecomendations['Restaurante'].items():
    print(name)
print('-------------La información contenida en los primeros',num_sv,'valores es:------------------------')
SVSSum=round((SVD.explained_variance_ratio_).sum()*100,2)
print(SVSSum,'%')

----------------El Número de valores singulares mínimo para encontrar 9 de las 10 recomendaciones es:-----------------------
24
-------------Las recomendaciones enctontradas son-----------------------
Rincon del Bife
Restaurant Teely
Paniroles
Chilis Cuernavaca
Restaurant and Bar and Clothesline Carlos N Charlies
Vips
Mariscos Tia Licha
Log Yin
Sanborns Casa Piedra
Mikasa
-------------La informacion contenida en los primeros 24 valores es:------------------------
68.35 %


In [ ]:
LTmp = DFCompare.sort_values(by='Restaurante',ascending=True)
ListA=LTmp['Restaurante'].values.tolist()
print('Primera recomendación: ')
print(ListA)
LTmp = firstTenRecomendations.sort_values(by='Restaurante',ascending=True)
ListB=LTmp['Restaurante'].values.tolist()
print('Segunda recomendación: ')
print(ListB)
print('Diferencia entre recomendaciones:')
print(list(set(ListA) - set(ListB)),list(set(ListB) - set(ListA)))

Primera recomendación: 
['Chilis Cuernavaca', 'Log Yin', 'Mariscos Tia Licha', 'Mikasa', 'Restaurant Teely', 'Restaurant and Bar and Clothesline Carlos N Charlies', 'Restaurant los Pinos', 'Rincon del Bife', 'Sanborns Casa Piedra', 'Vips']
Segunda recomendación: 
['Chilis Cuernavaca', 'Log Yin', 'Mariscos Tia Licha', 'Mikasa', 'Paniroles', 'Restaurant Teely', 'Restaurant and Bar and Clothesline Carlos N Charlies', 'Rincon del Bife', 'Sanborns Casa Piedra', 'Vips']
Diferencia entre recomendaciones:
['Restaurant los Pinos'] ['Paniroles']


#### Ejercicio-5.
Repite el ejercicio con el restaurante "tacos de barbacoa enfrente del Tec". Incluye tus conclusiones. 

In [ ]:
RestLabel = 'tacos de barbacoa enfrente del Tec'
Corr = corrPearson[restaurantes.index(RestLabel)]
sTmp = Corr.reshape((-1,1))
rTmp = np.array(restaurantes).reshape((-1,1))
aTmp = np.concatenate( (rTmp,sTmp),axis=1 )
CorrLabel = 'Corr_'+RestLabel
CorrDf = pd.DataFrame(aTmp,columns=['Restaurante',CorrLabel])
CorrDf.head(2)
firstTenRecomendationsEx5 = CorrDf.sort_values(by=CorrLabel,ascending=False)[1:11]
firstTenRecomendationsEx5

,Restaurante,Corr_tacos de barbacoa enfrente del Tec
128,vips,0.8119478180642272
123,tacos abi,0.6592592592592608
114,little pizza Emilio Portes Gil,0.6463244307679646
92,Taqueria EL amigo,0.42278365020160186
118,puesto de gorditas,0.3977058393342043
8,Carreton de Flautas y Migadas,0.3922893952445135
36,Little Cesarz,0.2254338180824853
116,palomo tec,0.16832093243282217
25,Hamburguesas La perica,0.16265001215808922
101,carnitas_mata,0.1516866822022362


In [ ]:
# Para encontrar la cantidad de valores singulars mínima para que coincidan 9 de 10 recomendaciones voy a iterar y revisar los valores
# de cada iteración hasta que tenga 2 de 130

CorrLabel = 'Corr_'+RestLabel

# Ahora vamos a descomponer la matriz de utilidad en sus valores singulares
DFCompare=firstTenRecomendationsEx5
UtilityMatrix=utilityMatrixEx2T

num_sv = 0
firstTenRecomendations = pd.DataFrame()
SVD = None

for n in range(2,130):

    SVD = TruncatedSVD(n_components=n, n_iter=1000, random_state=8)
    
    reducedUtilityMatrix = SVD.fit_transform(UtilityMatrix)
    #Ahora calculamos la matriz de correlación usando la matriz reconstruida
    reducedUtilityMatrixCorr = np.corrcoef(reducedUtilityMatrix, )
    
    Corr = reducedUtilityMatrixCorr[restaurantes.index(RestLabel)]
    
    sTmp = Corr.reshape((-1,1))
    rTmp = np.array(restaurantes).reshape((-1,1))
    aTmp = np.concatenate( (rTmp,sTmp),axis=1 )
    
    CorrDf = pd.DataFrame(aTmp,columns=['Restaurante',CorrLabel])
    
    firstTenRecomendations = CorrDf.sort_values(by=CorrLabel,ascending=False)[1:11]
    
    strikes = 0
    for idx,name in firstTenRecomendations['Restaurante'].items():
        if name not in DFCompare['Restaurante'].unique():
            strikes += 1
        if strikes > 1:
            break
    if strikes == 1:
        num_sv = n
        break

print('----------------El Número de valores singulares mínimo para encontrar 9 de las 10 recomendaciones es:-----------------------')
print(num_sv)
print('-------------Las recomendaciones encontradas son-----------------------')
for idx,name in firstTenRecomendations['Restaurante'].items():
    print(name)
print('-------------La información contenida en los primeros',num_sv,'valores es:------------------------')
SVSSum=round((SVD.explained_variance_ratio_).sum()*100,2)
print(SVSSum,'%')

----------------El Número de valores singulares mínimo para encontrar 9 de las 10 recomendaciones es:-----------------------
33
-------------Las recomendaciones enctontradas son-----------------------
tacos abi
Taqueria EL amigo 
Little Cesarz
Hamburguesas La perica
vips
palomo tec
carnitas_mata
little pizza Emilio Portes Gil
churchs
Carreton de Flautas y Migadas
-------------La informacion contenida en los primeros 33 valores es:------------------------
77.74 %


In [ ]:
LTmp = DFCompare.sort_values(by='Restaurante',ascending=True)
ListA=LTmp['Restaurante'].values.tolist()
print('Primera recomendación: ')
print(ListA)
LTmp = firstTenRecomendations.sort_values(by='Restaurante',ascending=True)
ListB=LTmp['Restaurante'].values.tolist()
print('Segunda recomendación: ')
print(ListB)
print('Diferencia entre recomendaciones:')
print(list(set(ListA) - set(ListB)),list(set(ListB) - set(ListA)))

Primera recomendación: 
['Carreton de Flautas y Migadas', 'Hamburguesas La perica', 'Little Cesarz', 'Taqueria EL amigo ', 'carnitas_mata', 'little pizza Emilio Portes Gil', 'palomo tec', 'puesto de gorditas', 'tacos abi', 'vips']
Segunda recomendación: 
['Carreton de Flautas y Migadas', 'Hamburguesas La perica', 'Little Cesarz', 'Taqueria EL amigo ', 'carnitas_mata', 'churchs', 'little pizza Emilio Portes Gil', 'palomo tec', 'tacos abi', 'vips']
Diferencia entre recomendaciones:
['puesto de gorditas'] ['churchs']


Para lograr el 90% de similariad entre las recomendaciones utilizando el 100% de valores singulares con el restaurante "Subway" se logró unicamente con 3 de ellos, en comparación con el restaurante "tacos de barbacoa enfrente del Tec" que necesitó de 33 valores.

Lo que se puede concluir que el restaurante "Subway" tiene menos características sigulares o en su defecto, tiene mayores elementos comúnes con todos los restaurantes referenciados en este Database. 

En contraparte, el restaurante "tacos de barbacoa enfrente del Tec" tiene mayores caracteristicas singulares y comparte menos elementos con todos los restaurantes referenciados en este Database. 

#**Ejercicio-6.**

 Descarga los archivos y genera la matriz de utilidad en la cual los renglones son los usuarios 
(user_ID), las columnas los nombres de los restaurantes (name) y las entradas la evaluaciones 
generales dadas por los usuarios (rating). En particular indica el tamaño de dicha matriz y despliega 
sus primeros 10 renglones. Incluye tus conclusiones. 

In [ ]:
ratingDf = pd.read_csv(ratingFileName)
geoplacesDf = pd.read_csv(geoplacesFileName, encoding = "latin-1")
X6 = pd.merge(ratingDf, geoplacesDf, how="inner", on="placeID")[['userID', 'name', 'food_rating']]
utilityMatrixEx6 = X6.pivot_table(values='food_rating', index='userID', columns='name', fill_value=0)
print('------------Tamaño de Matriz--------------------')
print(utilityMatrixEx6.shape)
print('------------Primeros 10 renglones--------------------')
utilityMatrixEx6.head(10)

------------Tamaño de Matriz--------------------
(138, 129)
------------Primeros 10 renglones--------------------


name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,0,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,2,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
# Como queremos obtener información sobre las restuarantes, tenemos que dejar los renglones con los restaurantes y las columnas con los usuarios, para esto pudemos usar la matriz transpuesta
utilityMatrixEx6T = utilityMatrixEx6.T
utilityMatrixEx6T.shape

(129, 138)

In [ ]:
SVD6 = TruncatedSVD(n_components=137)
SVD6.fit(utilityMatrixEx6T)
print('------------El Numero de Valores singulares es--------------------')
print(SVD6.singular_values_.shape[0])

------------El Numero de Valores singulares es--------------------
129


In [ ]:
corrPearson = np.corrcoef(utilityMatrixEx6T, )

In [ ]:
# Ahora encontramos el número de renglón para poder ver la correlación con los otros restaurantes
RestLabel = 'Subway'

In [ ]:
# Creamos un dataframe con en nombre del Restaurante y la correlación al Subway
Corr = corrPearson[restaurantes.index(RestLabel)]
sTmp = Corr.reshape((-1,1))
rTmp = np.array(restaurantes).reshape((-1,1))
aTmp = np.concatenate( (rTmp,sTmp),axis=1 )
CorrLabel = 'Corr_'+RestLabel
CorrDf = pd.DataFrame(aTmp,columns=['Restaurante',CorrLabel])
CorrDf.head(2)

,Restaurante,Corr_Subway
0,Abondance Restaurante Bar,-0.037602134498118056
1,Arrachela Grill,-0.024836615736600107


In [ ]:
# Ahora encontraremos las 10 correlaciones más altas sin incluir Subway:
firstTenRecomendationsEx6 = CorrDf.sort_values(by=CorrLabel,ascending=False)[1:11]
firstTenRecomendationsEx6

,Restaurante,Corr_Subway
58,Restaurant Teely,0.8121238302041046
60,Restaurant and Bar and Clothesline Carlos N Ch...,0.4567527621199486
97,Vips,0.4211409395973146
41,Mariscos Tia Licha,0.37946587563502654
85,Rincon del Bife,0.36799237573694654
86,Sanborns Casa Piedra,0.33876487010086664
55,Restaurant Las Mananitas,0.2997589807943298
46,Paniroles,0.2947170778717423
29,Kiku Cuernavaca,0.22366624008488603
21,Giovannis,0.2087740338626092


In [ ]:
# Para encontrar la cantidad de valores singulars mínima para que coincidan 9 de 10 recomendaciones voy a iterar y revisar los valores
# de cada iteración hasta que tenga 2 de 130


# Ahora vamos a descomponer la matriz de utilidad en sus valores singulares
CorrLabel = 'Corr_'+RestLabel

# Ahora vamos a descomponer la matriz de utilidad en sus valores singulares
DFCompare=firstTenRecomendationsEx6
UtilityMatrix=utilityMatrixEx6T

num_sv = 0
firstTenRecomendations = pd.DataFrame()
SVD = None

for n in range(2,130):

    SVD = TruncatedSVD(n_components=n, n_iter=1000, random_state=8)
    
    reducedUtilityMatrix = SVD.fit_transform(UtilityMatrix)
    #Ahora calculamos la matriz de correlación usando la matriz reconstruida
    reducedUtilityMatrixCorr = np.corrcoef(reducedUtilityMatrix, )
    
    Corr = reducedUtilityMatrixCorr[restaurantes.index(RestLabel)]
    
    sTmp = Corr.reshape((-1,1))
    rTmp = np.array(restaurantes).reshape((-1,1))
    aTmp = np.concatenate( (rTmp,sTmp),axis=1 )
    
    CorrDf = pd.DataFrame(aTmp,columns=['Restaurante',CorrLabel])
    
    firstTenRecomendations = CorrDf.sort_values(by=CorrLabel,ascending=False)[1:11]
    
    strikes = 0
    for idx,name in firstTenRecomendations['Restaurante'].items():
        if name not in DFCompare['Restaurante'].unique():
            strikes += 1
        if strikes > 1:
            break
    if strikes == 1:
        num_sv = n
        break

print('----------------El Número de valores singulares mínimo para encontrar 9 de las 10 recomendaciones es:-----------------------')
print(num_sv)
print('-------------Las recomendaciones encontradas son-----------------------')
for idx,name in firstTenRecomendations['Restaurante'].items():
    print(name)
print('-------------La información contenida en los primeros',num_sv,'valores es:------------------------')
SVSSum=round((SVD.explained_variance_ratio_).sum()*100,2)
print(SVSSum,'%')

----------------El Número de valores singulares mínimo para encontrar 9 de las 10 recomendaciones es:-----------------------
62
-------------Las recomendaciones enctontradas son-----------------------
Restaurant Teely
Restaurant and Bar and Clothesline Carlos N Charlies
Rincon del Bife
Vips
Mariscos Tia Licha
Sanborns Casa Piedra
Restaurant Las Mananitas
Paniroles
Mikasa
Giovannis
-------------La informacion contenida en los primeros 62 valores es:------------------------
93.9 %


In [ ]:
LTmp = DFCompare.sort_values(by='Restaurante',ascending=True)
ListA=LTmp['Restaurante'].values.tolist()
print('Primera recomendación: ')
print(ListA)
LTmp = firstTenRecomendations.sort_values(by='Restaurante',ascending=True)
ListB=LTmp['Restaurante'].values.tolist()
print('Segunda recomendación: ')
print(ListB)
print('Diferencia entre recomendaciones:')
print(list(set(ListA) - set(ListB)),list(set(ListB) - set(ListA)))

Primera recomendación: 
['Giovannis', 'Kiku Cuernavaca', 'Mariscos Tia Licha', 'Paniroles', 'Restaurant Las Mananitas', 'Restaurant Teely', 'Restaurant and Bar and Clothesline Carlos N Charlies', 'Rincon del Bife', 'Sanborns Casa Piedra', 'Vips']
Segunda recomendación: 
['Giovannis', 'Mariscos Tia Licha', 'Mikasa', 'Paniroles', 'Restaurant Las Mananitas', 'Restaurant Teely', 'Restaurant and Bar and Clothesline Carlos N Charlies', 'Rincon del Bife', 'Sanborns Casa Piedra', 'Vips']
Diferencia entre recomendaciones:
['Kiku Cuernavaca'] ['Mikasa']


Para el presente caso, dado que se desea obtener la matriz de utilidad derivada de la entrada de evaluación rating, se observa que, como mínimo, se deben utilizar 62 valores singulares para alcanzar dicho análisis. Sin embargo, al contener esa cantidad de características se observa que las mismas contienen la información de casi el 94% del conjunto de datos original, lo que respresenta una mejora notable en el procesamiento del modelo puesto que no será necesario utilizar la totalidad de la muestra.

Respecto a lo anterior, el modelo genera 2 recomendaciones de 10 restaurantes en las que sólo Kiku Cuernavaca de la primer recomendación y Mikasa de la segunda son aquellas que difieren entre las recomendaciones.

#**Ejercicio-7.**

Repite los ejercicios 1 a 4 generando ahora la matriz de utilidad en la cual los renglones son los 
usuarios (user_ID), las columnas los nombres de los restaurantes (name) y las entradas la 
evaluación dada al servicio (service_rating). Incluye tus conclusiones. 

In [ ]:
ratingDf = pd.read_csv(ratingFileName)
geoplacesDf = pd.read_csv(geoplacesFileName, encoding = "latin-1")
X7 = pd.merge(ratingDf, geoplacesDf, how="inner", on="placeID")[['userID', 'name', 'service_rating']]
utilityMatrixEx7 = X7.pivot_table(values='service_rating', index='userID', columns='name', fill_value=0)
print('------------Tamaño de Matriz--------------------')
print(utilityMatrixEx7.shape)
print('------------Primeros 10 renglones--------------------')
utilityMatrixEx7.head(10)

------------Tamaño de Matriz--------------------
(138, 129)
------------Primeros 10 renglones--------------------


name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,1,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Como queremos obtener información sobre las restuarantes, tenemos que dejar los renglones con los restaurantes y las columnas con los usuarios, para esto pudemos usar la matriz transpuesta
utilityMatrixEx7T = utilityMatrixEx7.T
utilityMatrixEx7T.shape

(129, 138)

In [ ]:
SVD7 =TruncatedSVD(n_components=137)
SVD7.fit(utilityMatrixEx7T)
print('------------El Numero de Valores singulares es--------------------')
print(SVD7.singular_values_.shape[0])

------------El Numero de Valores singulares es--------------------
129


In [ ]:
corrPearson = np.corrcoef(utilityMatrixEx7T, )

In [ ]:
# Ahora encontramos el número de renglón para poder ver la correlación con los otros restaurantes
RestLabel = 'Subway'

In [ ]:
# Creamos un dataframe con en nombre del Restaurante y la correlación al Subway
Corr = corrPearson[restaurantes.index(RestLabel)]
sTmp = Corr.reshape((-1,1))
rTmp = np.array(restaurantes).reshape((-1,1))
aTmp = np.concatenate( (rTmp,sTmp),axis=1 )
CorrLabel = 'Corr_'+RestLabel
CorrDf = pd.DataFrame(aTmp,columns=['Restaurante',CorrLabel])
CorrDf.head(2)

,Restaurante,Corr_Subway
0,Abondance Restaurante Bar,-0.038856684529147215
1,Arrachela Grill,-0.01881478737696169


In [ ]:
# Ahora encontraremos las 10 correlaciones más altas sin incluir Subway:
firstTenRecomendationsEx7 = CorrDf.sort_values(by=CorrLabel,ascending=False)[1:11]
firstTenRecomendationsEx7

,Restaurante,Corr_Subway
58,Restaurant Teely,0.7696236410799917
60,Restaurant and Bar and Clothesline Carlos N Ch...,0.5892857142857143
85,Rincon del Bife,0.5054244807092463
97,Vips,0.39442769090681523
46,Paniroles,0.3496975261398902
11,Chilis Cuernavaca,0.20839459883482936
55,Restaurant Las Mananitas,0.19284951144852297
21,Giovannis,0.15279829472295042
54,Restaurant Familiar El Chino,0.15279829472295028
29,Kiku Cuernavaca,0.1438953555739948


In [ ]:
CorrLabel = 'Corr_'+RestLabel

# Ahora vamos a descomponer la matriz de utilidad en sus valores singulares
DFCompare=firstTenRecomendationsEx7
UtilityMatrix=utilityMatrixEx7T

num_sv = 0
firstTenRecomendations = pd.DataFrame()
SVD = None

for n in range(2,130):

    SVD = TruncatedSVD(n_components=n, n_iter=1000, random_state=8)
    
    reducedUtilityMatrix = SVD.fit_transform(UtilityMatrix)
    #Ahora calculamos la matriz de correlación usando la matriz reconstruida
    reducedUtilityMatrixCorr = np.corrcoef(reducedUtilityMatrix, )
    
    Corr = reducedUtilityMatrixCorr[restaurantes.index(RestLabel)]
    
    sTmp = Corr.reshape((-1,1))
    rTmp = np.array(restaurantes).reshape((-1,1))
    aTmp = np.concatenate( (rTmp,sTmp),axis=1 )
    
    CorrDf = pd.DataFrame(aTmp,columns=['Restaurante',CorrLabel])
    
    firstTenRecomendations = CorrDf.sort_values(by=CorrLabel,ascending=False)[1:11]
    
    strikes = 0
    for idx,name in firstTenRecomendations['Restaurante'].items():
        if name not in DFCompare['Restaurante'].unique():
            strikes += 1
        if strikes > 1:
            break
    if strikes == 1:
        num_sv = n
        break

print('----------------El Número de valores singulares mínimo para encontrar 9 de las 10 recomendaciones es:-----------------------')
print(num_sv)
print('-------------Las recomendaciones encontradas son-----------------------')
for idx,name in firstTenRecomendations['Restaurante'].items():
    print(name)
print('-------------La información contenida en los primeros',num_sv,'valores es:------------------------')
SVSSum=round((SVD.explained_variance_ratio_).sum()*100,2)
print(SVSSum,'%')

----------------El Número de valores singulares mínimo para encontrar 9 de las 10 recomendaciones es:-----------------------
47
-------------Las recomendaciones enctontradas son-----------------------
Subway
Restaurant Teely
Restaurant and Bar and Clothesline Carlos N Charlies
Rincon del Bife
Paniroles
Vips
Giovannis
Restaurant Las Mananitas
Chilis Cuernavaca
Kiku Cuernavaca
-------------La informacion contenida en los primeros 47 valores es:------------------------
87.67 %


In [ ]:
LTmp = DFCompare.sort_values(by='Restaurante',ascending=True)
ListA=LTmp['Restaurante'].values.tolist()
print('Primera recomendación: ')
print(ListA)
LTmp = firstTenRecomendations.sort_values(by='Restaurante',ascending=True)
ListB=LTmp['Restaurante'].values.tolist()
print('Segunda recomendación: ')
print(ListB)
print('Diferencia entre recomendaciones:')
print(list(set(ListA) - set(ListB)),list(set(ListB) - set(ListA)))

Primera recomendación: 
['Chilis Cuernavaca', 'Giovannis', 'Kiku Cuernavaca', 'Paniroles', 'Restaurant Familiar El Chino', 'Restaurant Las Mananitas', 'Restaurant Teely', 'Restaurant and Bar and Clothesline Carlos N Charlies', 'Rincon del Bife', 'Vips']
Segunda recomendación: 
['Chilis Cuernavaca', 'Giovannis', 'Kiku Cuernavaca', 'Paniroles', 'Restaurant Las Mananitas', 'Restaurant Teely', 'Restaurant and Bar and Clothesline Carlos N Charlies', 'Rincon del Bife', 'Subway', 'Vips']
Diferencia entre recomendaciones:
['Restaurant Familiar El Chino'] ['Subway']


Con respecto a los numerales anteriores, para el caso de service_rating se obtuvo una información contenida del 87,67%. Derivado de esto, se encontró que existen 47 valores singulares que explican dicha similaridad con respecto a que se utiliza como variable de comparación el service_rating.

De acuerdo a lo anterior se puede observar que, al momento de realizar el modelo, se utilizan más features o variables mínimas para encontrar, en este caso las recomendaciones que se desean obtener; como se ha observado en los numerales anteriores, cuando se utilizan pocas variables en la obtención de la variabilidad, dicha medida se reduce, sin embargo, dicha variabilidad se incrementa cuando se utilizaron más variables en su obtención.

#**Ejercicio-8.**

Incluye tus conclusiones finales. En particular indica la coincidencia o no de estos tres modelos. 

El método de la reducción de la dimensionalidad posee grandes ventajas para el aprendizaje automático del modelo gracias a que permite acortar la cantidad de variables que se usan para las predicciones esperadas. Lo anterior se presenta dado que, en un dataset, pueden existir características excesivas o repetitivas que pueden no ser necesarias de incluir y podrían ser sesgadas por pocas features que contengan dicha información. Por tanto, cuando se tienen conjuntos de datos que contienen un elevado número de características de entrada (como el que se ha usado en el ejercicio), la reducción de la dimensionalidad genera un conjunto de datos de dimensiones menores, asegurando que la información que proporciona es similar a la original, simplificando la complejidad del tratamiento de los datos y el procesamiento del modelo.

A lo largo del ejercicio se ha podido observar que la reducción de la dimensionalidad permite aminorar la cantidad de variables utilizadas en un modelo para predecir su comportamiento. Esto a partir de ciertas cualidades que se le indican al mismo, es decir, personalizar el modelo de acuerdo a features específicos.

Para el caso del dataset, en algunos de los numerales desarrollados se ha observado que, partiendo de un accuracy del 90% del modelo, se han encontrado cierta cantidad de variables que contienen o explican una proporción de la totalidad de los componentes del dataset.

Dentro del ejercicio, se muestra que, a medida que se utilizan más variables en la consecución de la predicción, aumenta la información contenida de otras variables; en otras palabras, existen características singulares que pueden explicar el comportamiento de otras, evitando utilizar la totalidad del dataset, reduciendo los tiempos de ejecución del modelo y la carga de máquina para las predicciones sugeridas del modelo. Lo anterior se observa a lo largo de los numerales 4, 6 y 7 que tienen dicho comportamiento, a partir de una cualidad sugerida para su análisis (las entradas de evaluación incluidas en la matriz de utilidad de cada ejercicio).

###**Fin de la Actividad de la semana 10 - Sistemas de Recomendación y Reducción de Dimensionalidad con SVD.**